In [ ]:
import numpy as np
import pandas as pd
import multiprocessing

import scipy
from scipy import spatial
from scipy.spatial import cKDTree

import sklearn as sk
from sklearn import svm
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn import clone
from sklearn.externals.six.moves import xrange

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline

%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

In [ ]:
# 2018: SS18/FW18 ranges

# SS18 = pd.read_excel('RangeSS18.xlsx', skiprows = 5) # **PROBLEM**
SS18 = pd.read_excel('SS18_range_data.xlsx') # Solution: Artem file
FW18 = pd.read_excel('RangeFW18.xlsx', skiprows = 5)

SS18_range = SS18['Article Number'].unique()
FW18_range = FW18['Article Number'].unique()

In [ ]:
# SS20 = (pd.read_excel('SS20Range.xlsx', skiprows = 6).
#        drop(['Unnamed: 0', 'vlookup'], axis = 1))

# SS20_all_articles = SS20['Article Number'].unique()

# SS20 = SS20[SS20['CO'] == 'Y']
# dat['CO'].fillna('N', inplace = True)
# dat['SS19 Ranged'].fillna('N', inplace = True)
# dat['FW19 Ranged'].fillna('N', inplace = True)

# dat.to_csv('dat_SS20_range.csv')

dat_SS20_range = pd.read_csv('dat_SS20_range.csv') # *** DATA ***
SS20_range = dat_SS20_range['Article Number'].unique() # 1340

In [ ]:
# EDA
dat_SS20_range = dat_SS20_range[['Article Number', 'Model', 'Product Division', 
                                 'Category Marketing Line', 'Gender', 'Model Number', 'Product Group',
                                 'Product Type', 'Sports Category', 'Key Category', 'WE eCom', 'SS19 Ranged', 
                                 'FW19 Ranged', 'CO']]


In [ ]:
dat_SS20_range.groupby(['SS19 Ranged', 'FW19 Ranged', 'CO'])['Article Number'].count()
# dat_SS20_range['Key Category'].value_counts()[:10]
# dat_SS20_range['Product Division'].value_counts()
# dat_SS20_range['Product Group'].value_counts()[:10]
# dat_SS20_range['Gender'].value_counts()

# dat_SS20_range[dat_SS20_range['FW19 Ranged'] == 'Y'].groupby('Article Number')[]

## Corrections

In [ ]:
# # ----- Benoit's magic ----
# dat.loc[dat.order_date.apply(lambda x: "mkt-Raiffeisen Bank" in x),:].index

In [ ]:
%%time

dat0 = pd.read_csv('Transaction data.csv', low_memory=False) # *** DATA ***
dat0['consumer_order_date'] = pd.to_datetime(dat0['consumer_order_date'])

In [83]:
dat = dat0.copy()

In [ ]:
# EDA: clearance transactions by season
dat.groupby('season')['Clearance'].value_counts()

In [84]:
# Transaction subset: SS19
dat = dat[['consumer_order_date', 'article_number', 'gross_demand_quantity', 'Clearance', 'season']]
dat = dat[dat['season'] == 'SS19']

In [85]:
# Wall time: 38.7 s

dat_SS20_range = pd.read_csv('dat_SS20_range.csv') # *** DATA ***
SS20_range = dat_SS20_range['Article Number'].unique()
dat = dat[[(a in SS20_range) for a in dat['article_number']]]

CPU times: user 38.5 s, sys: 351 ms, total: 38.9 s
Wall time: 38.7 s


In [86]:
clearance = dat.copy() # FOR CLEARANCE CORRECTION IN NEXT SECTION

# 'aggregate' to weekly sums by article for buy_availability merge and adjustment
dat.set_index('consumer_order_date', inplace = True)

dat = dat[['article_number', 'gross_demand_quantity']].groupby(['article_number']).resample('W').sum()
dat.reset_index(inplace=True)

# Add 'week' and 'year' for merging with stock (buy_availability) data (b/c min_date_of_week)
dat['week'] = [t.week for t in dat['consumer_order_date']]
dat['year'] = [t.year for t in dat['consumer_order_date']]

In [87]:
# Wall time: ~40 s

# 2018 data --> SS19 in Dec. 2018 -- to merge with 2019 SS19
dat_stock = pd.read_csv('Stock Data/Stock data 2018.csv', low_memory=False, index_col = 0) # *** DATA ***
dat_stock['min_date_of_week'] = pd.to_datetime(dat_stock['min_date_of_week'])
dat_stock = dat_stock[dat_stock['min_date_of_week'] > pd.to_datetime('2018-12-1')] # filter to > Dec1 (SS19)

# 2019 data
dat_stock2 = pd.read_csv('Stock Data/Stock data 2019.csv', low_memory=False, index_col = 0) # *** DATA ***

# Build: Dec2018 + 2019
dat_stock = pd.concat([dat_stock, dat_stock2])

# Tidy
dat_stock.reset_index(inplace = True)
dat_stock.drop(['avg(ecom_available_stock)', 'avg(size_availability)'], axis = 1, inplace = True)
dat_stock.rename(columns = {'min_date_of_week': 'date', 'avg(buy_availability)': 'buy_availability'}, inplace = True)

# Filter to SS20_range carryovers
dat_stock = dat_stock[[(a in SS20_range) for a in dat_stock['article_number']]]

dat_stock['date'] = pd.to_datetime(dat_stock['date'])

# For merging with transaction data
dat_stock['week'] = [t.week for t in dat_stock['date']]
dat_stock['year'] = [t.year for t in dat_stock['date']]

In [88]:
# weekly demand + buy_availability -- for understock correction
dat = pd.merge(dat, dat_stock, 
               left_on = ['article_number', 'year', 'week'], 
               right_on = ['article_number', 'year', 'week'],
               how = 'outer')

# dat['buy_availability'] = dat['buy_availability'].fillna(1)

In [89]:
# ----- Calculate *observed* full season gross_qty per article -----
dat_season = pd.DataFrame(dat.groupby(['article_number'])['gross_demand_quantity'].sum())

dat_season.rename(columns = {'gross_demand_quantity':'season_gross_demand_quantity'}, inplace = True)

In [90]:
# WEEKLY averages for articles when fully stocked
dat_stocked = pd.DataFrame(
    dat[dat['buy_availability'] > 0.35].
    groupby(['article_number'])['gross_demand_quantity'].
    mean())

dat_stocked.rename(columns = {'gross_demand_quantity':'stocked_weekly_avg_gross_demand_quantity'}, inplace= True)

dat_stocked['understock_correction'] = 26*dat_stocked['stocked_weekly_avg_gross_demand_quantity']

In [91]:
dat_season = pd.merge(dat_season, dat_stocked, left_index=True, right_index=True, how = 'outer')
dat_season.drop('stocked_weekly_avg_gross_demand_quantity', inplace=True, axis = 1)

In [ ]:
# ------ OVERSTOCK CORRECTION ------

In [92]:
clearance0 = clearance[clearance['Clearance'] == 0]

clearance0 = pd.DataFrame(clearance0.groupby(['article_number'])['gross_demand_quantity'].sum())

clearance0.rename(columns = {'gross_demand_quantity': 'overstock_correction'}, inplace = True)

In [ ]:
# EDA -- article intersection -- articles in (i) SS20 range AND (ii) in SS19 clearance transactions
set(SS20_range).intersection(set(clearance[clearance['Clearance'] == 1].reset_index()['article_number'])) # 37

In [93]:
dat_season = pd.merge(dat_season, clearance0, how = 'outer', left_index = True, right_index = True)

In [94]:
preds = (pd.DataFrame(
    pd.read_csv('Buyers predictions.csv', low_memory=False, index_col = 0)) # *** DATA ***
         [['season', 'ecom_marketing_forecast']].
         reset_index().
        dropna())

preds = preds[preds['season'] == 'SS19']

dat_season = pd.merge(
    dat_season, preds, 
    left_on = ['article_number'],
    right_on=['article'], 
    how = 'left').round()

dat_season = dat_season[['article', 'season', 'season_gross_demand_quantity', 'understock_correction',
                        'overstock_correction', 'ecom_marketing_forecast']]

In [95]:
def projection(gross, forecast, under, over):
    if forecast < gross:
        return under
    elif forecast >= gross:
        return 1.2*over
    else:
        return gross

dat_season.loc[:,'projected_gross_demand_quantity'] = (
    dat_season.apply(lambda row: projection(row['season_gross_demand_quantity'], 
                                            row['ecom_marketing_forecast'], 
                                            row['understock_correction'],
                                            row['overstock_correction']), 
                     axis = 1))

In [96]:
# Article subset: projected to have gross_demand_quantity > 100 in SS19
dat_season = dat_season[dat_season['season_gross_demand_quantity'] > 83]

In [106]:
dat_season['SS20_prediction'] = dat_season['projected_gross_demand_quantity']*1.1

In [107]:
dat_season.shape
dat_season

(491, 8)

,article,season,season_gross_demand_quantity,understock_correction,overstock_correction,ecom_marketing_forecast,projected_gross_demand_quantity,SS20_prediction
1,015110,SS19,280.0,383.0,280.0,800.0,336.0,369.60
2,019000,SS19,693.0,920.0,693.0,2000.0,831.6,914.76
3,019228,SS19,174.0,233.0,174.0,500.0,208.8,229.68
4,019310,SS19,129.0,163.0,129.0,500.0,154.8,170.28
7,033905,SS19,498.0,638.0,498.0,1500.0,597.6,657.36
10,280647,SS19,5800.0,7390.0,5800.0,40000.0,6960.0,7656.00
11,280648,SS19,1487.0,1967.0,1487.0,20000.0,1784.4,1962.84
12,288022,SS19,3626.0,4754.0,3626.0,25000.0,4351.2,4786.32
13,604432,SS19,343.0,472.0,343.0,650.0,411.6,452.76
15,619995,SS19,655.0,845.0,655.0,2080.0,786.0,864.60


In [108]:
dat_season.to_csv('article_baselines.csv')

In [105]:
print('Projected Gross Demand Quantity summary')
dat_season['projected_gross_demand_quantity'].describe()
print()
print('Summary: eCom forecast minus projected gross demand')
(dat_season['ecom_marketing_forecast'] - dat_season['projected_gross_demand_quantity']).describe()


Projected Gross Demand Quantity summary


count      491.000000
mean       912.824847
std       1947.244000
min         88.800000
25%        195.000000
50%        404.400000
75%        903.000000
max      30168.000000
Name: projected_gross_demand_quantity, dtype: float64


Summary: eCom forecast minus projected gross demand


count      489.000000
mean      1546.785276
std       3265.034626
min     -10313.000000
25%        245.200000
50%        708.600000
75%       1646.600000
max      33040.000000
dtype: float64

In [ ]:
# Note: get for Artem over/understock correction for 'B41674', CE2441
